In [1]:
import pandas as pd
import numpy as np
import requests
from dataclasses import dataclass
import hashlib

In [2]:
class House:
    street_address: str = None
    city: str = None
    state: str = None
    zip_code: str = None
    beds: float = None
    baths: float = None
    sq_ft: float = None
    parking: float = None
    parking_type: str = None
    lot_size: float = None
    home_type: str = None
    zestimate: str = None
    og_price: str = None

    def __repr__(self):
        return f"{self.street_address} {self.city}, {self.state} {self.zip_code}: {self.zestimate}"

    @property
    def detailed(self):
        detail_string = """Address: {address}
        Home Type: {home_type}
        Beds: {beds}
        Baths: {baths}
        SqFt: {sq_ft}
        Lot Size: {lot_size}
        Zestimate: {zestimate}
        Last Selling Price: {og_price}
        Parking Spaces: {parking}
        Parking Type: {parking_type}"""
        return detail_string.format(
            address=str(self),
            home_type=self.home_type,
            beds=self.beds,
            baths=self.baths,
            sq_ft=self.sq_ft,
            lot_size=self.lot_size,
            zestimate=self.zestimate,
            og_price=self.og_price,
            parking=self.parking,
            parking_type=self.parking_type
        )

    @property
    def hsh(self):
        m = hashlib.md5()
        m.update(str(self).encode())
        return m.hexdigest()

    @classmethod
    def from_dict(cls, dictionary):
        return cls(**dictionary)

    def as_dict(self):
        return {
            'street_address': self.street_address,
            'city': self.city,
            'state': self.state,
            'zip_code': self.zip_code,
            'beds': self.beds,
            'baths': self.baths,
            'zestimate': self.zestimate,
            'last_price': self.og_price,
            'parking': self.parking,
            'parking_type': self.parking_type,
            'sq_ft': self.sq_ft,
            'lot_size': self.lot_size,
            'home_type': self.home_type
        }

    def matches_search(self, beds=None, baths=None, sq_ft=None, parking=None, lot_size=None, price=None):
        return (
            (beds is None or self.beds >= beds) and
            (baths is None or self.baths >= baths) and
            (sq_ft is None or self.sq_ft >= sq_ft) and
            (parking is None or self.parking >= parking) and
            (lot_size is None or self.lot_size >= lot_size) and 
            (price is None or self.zestimate <= price)
        )

In [3]:
class Listing:
    """
    Listing class
    """

    def __init__(
        self,
        house=None,
        list_price=None,
        zestimate=None,
        days_on_market=None,
        original_list_price=None,
        status=None,
        mls_id=None,
        open_house_date=None,
        open_house_start_time=None,
        open_house_end_time=None
    ):
        self.house = house
        self.list_price = self._convert_to_numeric(list_price)
        self.zestimate = self._convert_to_numeric(zestimate)
        self.days_on_market = self._convert_to_numeric(days_on_market)
        self.original_list_price = self._convert_to_numeric(original_list_price)
        self.status = status
        self.mls_id = mls_id
        self.open_house_date = open_house_date
        self.open_house_start_time = open_house_start_time
        self.open_house_end_time = open_house_end_time

    def __repr__(self):
        return f"Address: {self.house} - List Price: {self.list_price} - Zestimate: {self.zestimate}"

    @property
    def detailed(self):
        detail_string = "House Details:\n{house_details}\nStatus: {status}\nList Price: {list_price}\nZestimate: {zestimate}\nMLS ID: {mls_id}\nDays on Market: {days_on_market}\nOriginal Price: {original_list_price}\nOpen House: {open_house_date} - {open_house_start_time} to {open_house_end_time}\n\n"
        return detail_string.format(
            house_details=self.house.detailed,
            status=self.status,
            list_price=self.list_price,
            zestimate=self.zestimate,
            mls_id=self.mls_id,
            days_on_market=self.days_on_market,
            original_list_price=self.original_list_price,
            open_house_date=self.open_house_date,
            open_house_start_time=self.open_house_start_time,
            open_house_end_time=self.open_house_end_time
        )

    def _convert_to_numeric(self, value):
        if isinstance(value, (int, float)):
            return value
        try:
            return float(value)
        except (TypeError, ValueError):
            return None

    @property
    def house(self):
        return self._house

    @house.setter
    def house(self, house):
        self._house = house

    @property
    def list_price(self):
        return self._list_price

    @list_price.setter
    def list_price(self, list_price):
        self._list_price = list_price

    @property
    def zestimate(self):
        return self._zestimate

    @zestimate.setter
    def zestimate(self, zestimate):
        self._zestimate = zestimate

    @property
    def days_on_market(self):
        return self._days_on_market

    @days_on_market.setter
    def days_on_market(self, days_on_market):
        self._days_on_market = days_on_market

    @property
    def original_list_price(self):
        return self._original_list_price

    @original_list_price.setter
    def original_list_price(self, original_list_price):
        self._original_list_price = original_list_price

    @property
    def status(self):
        return self._status

    @status.setter
    def status(self, status):
        self._status = status

    @property
    def mls_id(self):
        return self._mls_id

    @mls_id.setter
    def mls_id(self, mls_id):
        self._mls_id = mls_id

    @property
    def open_house_date(self):
        return self._open_house_date

    @open_house_date.setter
    def open_house_date(self, open_house_date):
        self._open_house_date = open_house_date

    @property
    def open_house_start_time(self):
        return self._open_house_start_time

    @open_house_start_time.setter
    def open_house_start_time(self, open_house_start_time):
        self._open_house_start_time = open_house_start_time

    @property
    def open_house_end_time(self):
        return self._open_house_end_time

    @open_house_end_time.setter
    def open_house_end_time(self, open_house_end_time):
        self._open_house_end_time = open_house_end_time

    @property
    def hsh(self):
        return self.house.hsh

    @classmethod
    def from_dict(cls, dictionary):
        try:
            h = House.from_dict(dictionary['house'])
            return cls(
                house=h,
                list_price=dictionary['list_price'],
                zestimate=dictionary['zestimate'],
                days_on_market=dictionary['days_on_market'],
                original_list_price=dictionary['original_list_price'],
                status=dictionary['status'],
                mls_id=dictionary['mls_id'],
                open_house_date=dictionary['open_house_date'],
                open_house_start_time=dictionary['open_house_start_time'],
                open_house_end_time=dictionary['open_house_end_time']
            )
        except (KeyError, TypeError):
            return cls()

    def as_dict(self):
        d = {
            'house': self.house.as_dict(),
            'list_price': self.list_price,
            'zestimate': self.zestimate,
            'days_on_market': self.days_on_market,
            'original_list_price': self.original_list_price,
            'status': self.status,
            'mls_id': self.mls_id,
            'open_house_date': self.open_house_date,
            'open_house_start_time': self.open_house_start_time,
            'open_house_end_time': self.open_house_end_time
        }
        return d
##################################################################################################
    def get_zestimate(self):
        lc = ListCache()
        lc.remove_old_listings()
        if lc.listing_in_cache(self):
            c_list = lc.retrieve_listing(self)
            self.zestimate = c_list.zestimate
        else:
            z_api = ZillAPI()
            z_list = z_api.get_from_zillow(self.house)
            self.zestimate = z_api.get_zestimate(z_list)
            lc.insert_listing(self)
####################################################################################################
    def matches_search(
        self,
        house=None,
        list_price=None,
        zestimate=None,
        days_on_market=None,
        status=None
    ):
        if (
            (list_price is None or self.list_price <= list_price) and
            (zestimate is None or self.zestimate <= zestimate) and
            (days_on_market is None or self.days_on_market <= days_on_market)
        ):
            if status is not None and self.status != status:
                return False
            return True
        return False

    def as_html(self):
        html_string = "<tr>"
        for key in self.as_dict():
            if key == 'house':
                h = House.from_dict(self.as_dict()[key])
                for key2 in h.as_dict():
                    html_string += "<td>%s</td>" % (h.as_dict()[key2])
            else:
                html_string += "<td>%s</td>" % (self.as_dict()[key])
        html_string += "</tr>"
        return html_string

    def html_headers(self):
        html_string = "<tr>"
        for key in self.as_dict():
            if key == 'house':
                h = House.from_dict(self.as_dict()[key])
                for key2 in h.as_dict():
                    html_string += "<th>%s</th>" % (key2)
            else:
                html_string += "<th>%s</th>" % (key)
        html_string += "</tr>"
        return html_string


In [41]:
from redfin import Redfin

In [42]:
def get_info_from_address(address):
    client = Redfin()
    response = client.search(address)
    house = House()
    try:
        url = response['payload']['exactMatch']['url']
    except:
        return house
    initial_info = client.initial_info(url)
    property_id = initial_info['payload']['propertyId']
    listing_id = initial_info['payload']['listingId']
    
    mls = client.below_the_fold(property_id)['payload']
    info = client.avm_details(property_id, listing_id)['payload']
    
    
    
    restimate = info['sectionPreviewText']
    beds = info['numBeds']
    baths = info['numBaths']
    
    sqft = info['sqFt']['value']
        
    
    string = url.strip("/")
    parts = string.split("/")

    state = parts[0]
    city = parts[1].replace("-", " ")
    initial = parts[2].split("-")
    zip_code = initial[-1]
    initial.pop()
    addy = " ".join(initial)

    
    try: 
        land_sq_ft = mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][3]['amenityEntries'][1]['amenityValues'][0]
    except:
        land_sq_ft = None
    try:  
        parking_spaces =  mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][2]['amenityEntries'][0]['amenityValues'][0]
    except:
        parking_spaces = None
    
    try:
        parking_type = mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][2]['amenityEntries'][1]['amenityValues'][0]
    except:
        parking_type = None
    
    try: 
        home_type = mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][0]['amenityEntries'][3]['amenityValues'][0]
    except: 
        home_type = "New"
    try:
        og_price = info['lastSoldPrice']
    except:
        og_price = "N/A"
    
    
    house.street_address = addy; house.city = city; house.state = state; house.zip_code = zip_code; house.beds = beds; house.baths = baths;
    house.zestimate = restimate; house.og_price = og_price; house.sq_ft = sqft; house.parking = parking_spaces; house.lot_size = land_sq_ft; house.parking_type = parking_type; house.home_type = home_type
    return house


In [43]:
print(get_info_from_address("2 Buttons Rd, Chapel Hill, NC").detailed)

Address: 2 Buttons Rd Chapel Hill, NC 27514: $1,300,323
        Home Type: Hip
        Beds: 5
        Baths: 3.0
        SqFt: 3594
        Lot Size: 30,928
        Zestimate: $1,300,323
        Last Selling Price: N/A
        Parking Spaces: ROCKY RIDGE
        Parking Type: 3,399


In [48]:
pip install numpy_financial

In [53]:
import numpy_financial as npf
import re

In [125]:
class Homes:
    def __init__(self):
        self.house_data = pd.DataFrame()
        self.reet = pd.DataFrame()
    
    def __repr__(self):
        return self.house_data.to_string()
    
    #Adds a House class object to Homes
    def add_house(self, house):
        
        self.house_data = pd.concat([self.data, pd.DataFrame([house.as_dict()])], ignore_index=True)
    
    #Queries address, makes it a House object then inserts the house into Homes
    def add_address(self, address):
        house = get_info_from_address(address)
        self.add_house(house)
        
    
    #Removes house based on name
    def remove_house(self, house_name):
        self.house_data = self.house_data[self.house_data['street_address'] != house_name].reset_index(drop=True)
        
    #Returns the dataframe of the data
    def get_data(self):
        return self.house_data
    
    #Returns reet dataframe
    def get_reet(self):
        return self.reet
    
    #Goes through data and removes the data which had a faulty URL and returned as None;
    #Doesn't remove new houses which have None for certain columns
    def clean_house_data(self):
        self.house_data.dropna()
        mask = self.house_data.isna().all(axis=1)
        new_mask = self.house_data['home_type'].str.contains('New', case=False, na=False)
        remove_mask = mask & ~new_mask
        self.data = self.house_data[~remove_mask].reset_index(drop=True)
    
    
    def add_info_from_address(self, address):
        client = Redfin()
        response = client.search(address)
        house = House()
        try:
            url = response['payload']['exactMatch']['url']
            initial_info = client.initial_info(url); property_id = initial_info['payload']['propertyId']; listing_id = initial_info['payload']['listingId']
            mls = client.below_the_fold(property_id)['payload']
            info = client.avm_details(property_id, listing_id)['payload']
            coh = client.cost_of_home_ownership(property_id)['payload']['mortgageCalculatorInfo']
            rent = client.rental_estimate(property_id, listing_id)['payload']            
        except:
            return house


        restimate = info['sectionPreviewText']; beds = info['numBeds']; baths = info['numBaths']; sqft = info['sqFt']['value']
        
        string = url.strip("/"); parts = string.split("/")
        state = parts[0]; city = parts[1].replace("-", " "); initial = parts[2].split("-"); zip_code = initial[-1]; initial.pop(); addy = " ".join(initial)
        
        try: 
            land_sq_ft = mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][3]['amenityEntries'][1]['amenityValues'][0]
        except:
            land_sq_ft = None
        
        try:  
            parking_spaces =  mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][2]['amenityEntries'][0]['amenityValues'][0]
        except:
            parking_spaces = None
        
        try:
            parking_type = mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][2]['amenityEntries'][1]['amenityValues'][0]
        except:
            parking_type = None
        
        try: 
            home_type = mls['amenitiesInfo']['superGroups'][1]['amenityGroups'][0]['amenityEntries'][3]['amenityValues'][0]
        except: 
            home_type = "New"
        
        try:
            og_price = info['lastSoldPrice']
        except:
            og_price = "N/A"
        house = House()
        house.street_address = addy; house.city = city; house.state = state; house.zip_code = zip_code; house.beds = beds; house.baths = baths;
        house.zestimate = restimate; house.og_price = og_price; house.sq_ft = sqft; house.parking = parking_spaces; house.lot_size = land_sq_ft; house.parking_type = parking_type; house.home_type = home_type
        
        self.house_data = pd.concat([self.house_data, pd.DataFrame([house.as_dict()])], ignore_index=True)
        
        #REET PART
        ##########################################
        #Assuming these values, change if needed##
        ##########################################
        cc = 6000
        lt = 30
        price = float(info['predictedValue'])
        down = float(coh['downPaymentPercentage'])
        dp = price * down/100
        
        loan_cost = price - dp + cc
        ir = float(coh['mortgageRateInfo']['thirtyYearFixed'])
        mm = npf.pmt(ir/100/12, lt*12, -1 * loan_cost, 0)
        
        try: 
            hoa = re.findall(r'\d+', client.info_panel(property_id, listing_id)['payload']['mainHouseInfo']['selectedAmenities'][0]['content'])
            hoa = float(hoa[0])
        except:
            hoa = 0
            
        pt = float(coh['propertyTaxRate'])
        mpt = (price*(pt/100))/12
        mhi = float(coh['homeInsuranceRate'])/12/100 * price

        me = (mhi + mm + hoa + mpt)
        
        try:
            mr = float(rent['rentalEstimateInfo']['predictedValue'])
            cf = mr - me
        except:
            mr = 0
            cf = 0
        ar = mr * 12
        acf = cf * 12
        cr = ((ar - 12*(me-mm))/(price + cc))*100
        ccr = (acf/(dp + cc))*100
        reet = {
            'address': addy,
            'purchase_price': price,
            'down%': down,
            'down_payment': dp,
            'closing_costs': cc,
            'loan_amount': loan_cost,
            'loan_term': lt,
            'interest_rate': ir,
            'monthly_mortgage': mm,
            'monthly_hoa': hoa,
            'property_tax_rate': pt,
            'monthly_property_tax': mpt,
            'monthly_home_insurance': mhi,
            #'vacancy_rate': vr,
            'monthly_rent': mr,
            'annural_rent': ar,
            'monthly_expenses': me,
            'monthly_cash_flow': cf,
            'annual_cash_flow': acf,
            'cap_rate': cr,
            'cash_on_cash_return': ccr
        }
        self.reet = pd.concat([self.reet, pd.DataFrame([reet])], ignore_index=True)
        

In [126]:
new_one = Homes()

In [135]:
h_name = input("Enter an address please")

Enter an address please412 Constantine Court


In [138]:
new_one.add_info_from_address("23781 Golden Ridge Dr")

In [139]:
new_one.get_reet()

,address,purchase_price,down%,down_payment,closing_costs,loan_amount,loan_term,interest_rate,monthly_mortgage,monthly_hoa,property_tax_rate,monthly_property_tax,monthly_home_insurance,monthly_rent,annural_rent,monthly_expenses,monthly_cash_flow,annual_cash_flow,cap_rate,cash_on_cash_return
0,155 Manordale Dr,403333.27,20.0,80666.654,6000,328666.616,30,7.182,2226.946791,138.0,1.29,433.583265,157.972197,2078.0,24936.0,2956.502253,-878.502253,-10542.027040,3.953095,-12.163879
1,412 Constantine Ct,1242528.85,20.0,248505.770,6000,1000023.080,30,7.255,6825.312020,130.0,1.25,1294.300885,331.341027,4008.0,48096.0,8580.953932,-4572.953932,-54875.447185,2.164812,-21.561573
2,23781 Golden Ridge Dr,463657.17,20.0,92731.434,6000,376925.736,30,7.188,2555.466238,0.0,1.25,482.976219,123.641912,3333.0,39996.0,3162.084369,170.915631,2050.987578,6.966056,2.077340


In [108]:
new_one.get_data()

,street_address,city,state,zip_code,beds,baths,zestimate,last_price,parking,parking_type,sq_ft,lot_size,home_type
0,412 Constantine Ct,San Ramon,CA,94583,3,2.5,"$1,242,529 (+$523K since last sold)",719000,1,"2,130",1762,None,2
1,155 Manordale Dr,Chapel Hill,NC,27517,2,2.5,"$403,333 (+$137K since last sold)",265720,Attached Garage,Attached,1552,"1,437",New


In [13]:
new_one

   street_address     city state zip_code  beds  baths                        zestimate  last_price parking     parking_type  sq_ft lot_size      home_type
0  4544 Radnor St  Detroit    MI    48224     4    2.0  $58,317 (+$28K since last sold)       30000       1  Detached Garage    656    4,356  Single Family
1  4544 Radnor St  Detroit    MI    48224     4    2.0  $58,317 (+$28K since last sold)       30000       1  Detached Garage    656    4,356  Single Family

In [14]:
h_array = ["4544 Radnor St, Detroit, Michigan", "834 N Museo Dr, Mountain House, California"]
homes = Homes()
for house in h_array:
    homes.add_info_from_address(house)

homes

   street_address            city state zip_code  beds  baths                        zestimate last_price parking     parking_type  sq_ft lot_size      home_type
0  4544 Radnor St         Detroit    MI    48224     4    2.0  $58,317 (+$28K since last sold)      30000       1  Detached Garage    656    4,356  Single Family
1  834 N Museo Dr  Mountain House    CA    95391     3    2.5                         $908,496        N/A       2             None   2111    6,637            New

In [15]:
def select_by_condition(df, data, condition):
    fin_df = pd.DataFrame()
    try:
        if condition.startswith('='):
            condition_value = float(condition[1:])
            fin_df = df[df[data] == condition_value]
        elif condition.startswith('<='):
            condition_value = float(condition[2:])
            fin_df = df[df[data] <= condition_value]
        elif condition.startswith('<'):
            condition_value = float(condition[1:])
            fin_df = df[df[data] < condition_value]
        elif condition.startswith('>='):
            condition_value = float(condition[2:])
            fin_df = df[df[data] >= condition_value]
        elif condition.startswith('>'):
            condition_value = float(condition[1:])
            fin_df = df[df[data] > condition_value]
        else:
            fin_df = df[df[data] == condition]
            
    
    except:
        print("There was an issue with some of the data you entered")
        
    if fin_df.empty:
        print("There were no results =(")
    else:
        return fin_df

In [16]:
select_by_condition(homes.get_reet(), 'parking_type', "Detached Garage")

,street_address,city,state,zip_code,beds,baths,zestimate,last_price,parking,parking_type,sq_ft,lot_size,home_type
0,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family


In [17]:
homes.query_data()

,street_address,city,state,zip_code,beds,baths,zestimate,last_price,parking,parking_type,sq_ft,lot_size,home_type
0,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family
1,834 N Museo Dr,Mountain House,CA,95391,3,2.5,"$908,496",N/A,2,None,2111,"6,637",New


In [18]:
homes.add_address("4544 Radnor Street")

In [19]:
homes.query_data()

,street_address,city,state,zip_code,beds,baths,zestimate,last_price,parking,parking_type,sq_ft,lot_size,home_type
0,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family
1,834 N Museo Dr,Mountain House,CA,95391,3,2.5,"$908,496",N/A,2,None,2111,"6,637",New
2,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family


In [77]:
import time

In [140]:
h_array = ["4544 Radnor St, Detroit, Michigan", "184 W Viola St, Mountain House, California", "834 N Museo Dr, Mountain House, California", "2 Buttons Rd, Chapel Hill, NC", "5517 Barbee Chapel Rd", "217 Summerwalk Cir #0", "511 Hillsborough St", "120 Rolling Meadows Ln"]
i = 0
for house in h_array:
    new_one.add_info_from_address(house)
    i += 1
    if (i% 3 == 0):
        time.sleep(1.3)

In [141]:
new_one.get_reet()

,address,purchase_price,down%,down_payment,closing_costs,loan_amount,loan_term,interest_rate,monthly_mortgage,monthly_hoa,property_tax_rate,monthly_property_tax,monthly_home_insurance,monthly_rent,annural_rent,monthly_expenses,monthly_cash_flow,annual_cash_flow,cap_rate,cash_on_cash_return
0,155 Manordale Dr,403333.27,20.0,80666.654,6000,328666.616,30,7.182,2226.946791,138.0,1.29,433.583265,157.972197,2078.0,24936.0,2956.502253,-878.502253,-10542.027040,3.953095,-12.163879
1,412 Constantine Ct,1242528.85,20.0,248505.770,6000,1000023.080,30,7.255,6825.312020,130.0,1.25,1294.300885,331.341027,4008.0,48096.0,8580.953932,-4572.953932,-54875.447185,2.164812,-21.561573
2,23781 Golden Ridge Dr,463657.17,20.0,92731.434,6000,376925.736,30,7.188,2555.466238,0.0,1.25,482.976219,123.641912,3333.0,39996.0,3162.084369,170.915631,2050.987578,6.966056,2.077340
3,4544 Radnor St,58317.38,20.0,11663.476,6000,52653.904,30,7.204,357.551080,0.0,1.29,62.691183,25.270865,0.0,0.0,445.513129,0.000000,0.000000,-1.641150,0.000000
4,184 W Viola St,809959.33,20.0,161991.866,6000,653967.464,30,7.255,4463.428977,0.0,1.25,843.707635,215.989155,3089.0,37068.0,5523.125767,-2434.125767,-29209.509202,2.984418,-17.387454
5,834 N Museo Dr,908496.29,20.0,181699.258,6000,732797.032,30,7.188,4968.188414,0.0,1.25,946.350302,242.265677,3223.0,38676.0,6156.804393,-2933.804393,-35205.652718,2.669514,-18.756415
6,2 Buttons Rd,1300323.01,20.0,260064.602,6000,1046258.408,30,7.182,7089.134370,1.0,1.29,1397.847236,509.293179,2852.0,34224.0,8997.274785,-6145.274785,-73743.297421,0.867038,-27.716313
7,5517 Barbee Chapel Rd,159168.13,20.0,31833.626,6000,133334.504,30,7.170,902.352644,1.0,1.29,171.105740,62.340851,1635.0,19620.0,1136.799234,498.200766,5978.409187,10.175474,15.801840
8,511 Hillsborough St,745719.77,20.0,149143.954,6000,602575.816,30,7.182,4082.873691,330.0,1.29,801.648753,292.073577,3117.0,37404.0,5506.596020,-2389.596020,-28675.152244,2.703046,-18.482932


In [143]:
select_by_condition(new_one.get_reet(), 'cash_on_cash_return', ">=0")

,address,purchase_price,down%,down_payment,closing_costs,loan_amount,loan_term,interest_rate,monthly_mortgage,monthly_hoa,property_tax_rate,monthly_property_tax,monthly_home_insurance,monthly_rent,annural_rent,monthly_expenses,monthly_cash_flow,annual_cash_flow,cap_rate,cash_on_cash_return
2,23781 Golden Ridge Dr,463657.17,20.0,92731.434,6000,376925.736,30,7.188,2555.466238,0.0,1.25,482.976219,123.641912,3333.0,39996.0,3162.084369,170.915631,2050.987578,6.966056,2.07734
3,4544 Radnor St,58317.38,20.0,11663.476,6000,52653.904,30,7.204,357.551080,0.0,1.29,62.691183,25.270865,0.0,0.0,445.513129,0.000000,0.000000,-1.641150,0.00000
7,5517 Barbee Chapel Rd,159168.13,20.0,31833.626,6000,133334.504,30,7.170,902.352644,1.0,1.29,171.105740,62.340851,1635.0,19620.0,1136.799234,498.200766,5978.409187,10.175474,15.80184


In [24]:
homes.query_data()

,street_address,city,state,zip_code,beds,baths,zestimate,last_price,parking,parking_type,sq_ft,lot_size,home_type
0,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family
1,834 N Museo Dr,Mountain House,CA,95391,3,2.5,"$908,496",N/A,2,None,2111,"6,637",New
2,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family
3,4544 Radnor St,Detroit,MI,48224,4,2.0,"$58,317 (+$28K since last sold)",30000,1,Detached Garage,656,"4,356",Single Family
4,184 W Viola St,Mountain House,CA,95391,3,2.5,"$809,959",N/A,2,None,1957,"3,870",New
5,834 N Museo Dr,Mountain House,CA,95391,3,2.5,"$908,496",N/A,2,None,2111,"6,637",New
6,2 Buttons Rd,Chapel Hill,NC,27514,5,3.0,"$1,300,323",N/A,ROCKY RIDGE,"3,399",3594,"30,928",Hip
7,5517 Barbee Chapel Rd,Chapel Hill,NC,27517,2,1.0,"$159,168",N/A,Attached Carport,414,921,"24,786",Concrete Block
8,511 Hillsborough St,Chapel Hill,NC,27514,3,3.0,"$745,720 (+$295K since last sold)",450000,None,None,1780,None,New


# REET PART

------------------------------------------------------------------------------------------------------------------------

In [25]:
import redfin as Redfin
import pandas as pd
import time
import requests
from dataclasses import dataclass

In [ ]:
class REET:
    def __init__(self):
        self.data = pd.DataFrame()
    
    def __repr__(self):
        return self.data.to_string()
    
    def add_REET(address):
        data = get_REET(address)
        self.data = pd.concat([self.data, pd.DataFrame([house.as_dict()])], ignore_index=True)
    
    def get_REET(address):
        client = Redfin()
        response = client.search(address)
        house = House()
        try:
            url = response['payload']['exactMatch']['url']
        except:
            return None
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']
        listing_id = initial_info['payload']['listingId']

        mls = client.below_the_fold(property_id)['payload']
        info = client.avm_details(property_id, listing_id)['payload']